<a href="https://colab.research.google.com/github/nabilneal/DataScience/blob/main/SpeechDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, os
import pandas as pd
import json
import numpy as np
import re
pd.options.display.max_colwidth = 150 ###
file1 = r"share/racism.json"
file2 = r"share/sexism.json"
file3 = r"share/neither.json"

In [ ]:
tweetList = []
print("Started Reading JSON file which contains multiple JSON document")
with open(file3) as f:
    for jsonObj in f:
        tweetDict = json.loads(jsonObj)
        tweetList.append(tweetDict)

print("Printing each JSON Decoded Object")
count = 1
print(len(tweetList), len(tweetDict))
print(tweetDict)
dkeys = []
for k in tweetDict:
    dkeys.append(k)
print(dkeys)
# for tweet in tweetList:
#     print(count, " " , tweet["id_str"])
#     count+=1

Started Reading JSON file which contains multiple JSON document
Printing each JSON Decoded Object
11501 27
{'in_reply_to_user_id': None, 'id_str': '569363477095174145', 'is_quote_status': False, 'text': 'RT @TheMeninism: 😂 http://t.co/VQzuAXqNzd', 'coordinates': None, 'in_reply_to_status_id': None, 'retweeted_status': {'in_reply_to_user_id': None, 'id_str': '569361442455101440', 'is_quote_status': False, 'text': '😂 http://t.co/VQzuAXqNzd', 'coordinates': None, 'lang': 'und', 'in_reply_to_status_id': None, 'retweeted': False, 'retweet_count': 260, 'created_at': 'Sun Feb 22 05:02:00 +0000 2015', 'geo': None, 'extended_entities': {'media': [{'media_url_https': 'https://pbs.twimg.com/media/B-bHNxwIMAAKqld.jpg', 'type': 'photo', 'id_str': '569361442416373760', 'sizes': {'large': {'w': 599, 'resize': 'fit', 'h': 336}, 'thumb': {'w': 150, 'resize': 'crop', 'h': 150}, 'medium': {'w': 599, 'resize': 'fit', 'h': 336}, 'small': {'w': 340, 'resize': 'fit', 'h': 190}}, 'url': 'http://t.co/VQzuAXqNz

In [ ]:
df1 = pd.read_json(file1, lines=True)
df2 = pd.read_json(file2, lines=True)
df3 = pd.read_json(file3, lines=True)

In [ ]:
df1.columns

Index(['in_reply_to_screen_name', 'is_quote_status', 'Annotation', 'truncated',
       'id', 'place', 'favorite_count', 'favorited', 'retweeted',
       'in_reply_to_status_id_str', 'text', 'created_at', 'lang',
       'coordinates', 'in_reply_to_user_id', 'user', 'retweet_count',
       'in_reply_to_user_id_str', 'in_reply_to_status_id', 'source', 'id_str',
       'geo', 'contributors', 'entities', 'possibly_sensitive',
       'extended_entities', 'retweeted_status', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'withheld_in_countries'],
      dtype='object')

In [ ]:
print(df1.shape, df2.shape, df3.shape)

(1976, 31) (3430, 28) (11501, 32)


In [ ]:
df1_copy = df1[['text', 'Annotation']]
df2_copy = df2[['text', 'Annotation']]
df3_copy = df3[['text', 'Annotation']]

df3_copy

,text,Annotation
0,Someone is going home #mkr ...that obviously cannot cook !,none
1,They didn't even wash the chicken 😩 #MKR,none
2,#MKR Is honestly so fucking staged. The most over rated show after #ImACelebrityAU #MKR2015,none
3,Can someone smash that bottle of Rose &amp; Lime Cordial over Drasko's head please #MKR,none
4,"Will someone pls assist Colin in the washing of his hair. Sorry Colin, big fan but pls... Some shampoo mate! #mkr",none
...,...,...
11496,"RT @JakeM_1998: RT BillSpindle: It's all about power at the top, but for fighters on the ground #Tikrit is religious war on both sides …",none
11497,"RT @ThinkAgain_DOS: Iraq: #ISIS sets off 21 car bombs in Anbar – as usual, randomly killing &amp; maiming innocents. http://t.co/gufa754CRT htt…",none
11498,RT @ThePatriot143: DEAR STATE DEPARTMENT: WHERE IS HILLARY CLINTON’S OF-109 FORM http://t.co/aNkzFa5iVP #HillaryEmail #tcot #PJNET http://t…,none
11499,"""@panelrific: Let's go 🐧🐧🐧🐧🐧🐧😃""",none


In [ ]:
df1_copy

,text,Annotation
0,So Drasko just said he was impressed the girls cooked half a chicken.. They cooked a whole one #MKR,racism
1,Drasko they didn't cook half a bird you idiot #mkr,racism
2,Hopefully someone cooks Drasko in the next ep of #MKR,racism
3,of course you were born in serbia...you're as fucked as A Serbian Film #MKR,racism
4,"These girls are the equivalent of the irritating Asian girls a couple years ago. Well done, 7. #MKR",racism
...,...,...
1971,Drasko they didn't cook half a bird you idiot #mkr,racism
1972,Hopefully someone cooks Drasko in the next ep of #MKR,racism
1973,#MKR Lost the plot - where's the big Texan with the elephant sized steaks that they all have for brekkie ?,racism
1974,RT @Benfrancisallen: It hasn't been a good few weeks for #ISIS. A new front has opened up in #Sinjar and they're about to lose the battle f…,racism


In [ ]:
def clean_data(dataframe):
#replace URL of a text
    dataframe['text'] = dataframe['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    dataframe['text'] = dataframe['text'].str.replace('RT', ' ')
    dataframe['text'] = dataframe['text'].str.replace('@(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    dataframe['text'] = dataframe['text'].str.replace('[^\w\s#@/:%.,_-]', ' ')
    dataframe['Annotation'] = dataframe['Annotation'].str.replace('racism', '0')
    dataframe['Annotation'] = dataframe['Annotation'].str.replace('sexism', '1')
    dataframe['Annotation'] = dataframe['Annotation'].str.replace('none', '2')
clean_data(df1_copy)
clean_data(df2_copy)
clean_data(df3_copy)

C:\Users\nabil\AppData\Local\Temp\ipykernel_24224\422730061.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
C:\Users\nabil\AppData\Local\Temp\ipykernel_24224\422730061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['text'] = dataframe['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
C:\Users\nabil\AppData\Local\Temp\ipykernel_24224\422730061.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [ ]:
df_rows = pd.concat([df1_copy, df2_copy], ignore_index=True)
df_final = pd.concat([df_rows, df3_copy], ignore_index=True)

In [ ]:
df_final

,text,Annotation
0,So Drasko just said he was impressed the girls cooked half a chicken.. They cooked a whole one #MKR,0
1,Drasko they didn t cook half a bird you idiot #mkr,0
2,Hopefully someone cooks Drasko in the next ep of #MKR,0
3,of course you were born in serbia...you re as fucked as A Serbian Film #MKR,0
4,"These girls are the equivalent of the irritating Asian girls a couple years ago. Well done, 7. #MKR",0
...,...,...
16902,"BillSpindle: It s all about power at the top, but for fighters on the ground #Tikrit is religious war on both sides",2
16903,"Iraq: #ISIS sets off 21 car bombs in Anbar as usual, randomly killing amp maiming innocents. htt",2
16904,DEAR STATE DEPA MENT: WHERE IS HILLARY CLINTON S OF-109 FORM #HillaryEmail #tcot #PJNET,2
16905,Let s go,2


In [ ]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# test_df= df_final.sample(frac=0.25,random_state=1)
# training_df = df_final.drop(test_df.index)
# test_df
# training_df

In [ ]:
# train_X = training_df['text']
# train_y = training_df['Annotation']
# test_X = test_df['text']
# test_y = test_df['Annotation']

train_X,test_X, train_y, test_y = train_test_split(df_final['text'], df_final['Annotation'],
                                                   test_size = 0.2, random_state = 10, stratify = df_final['Annotation'])

In [ ]:
t = time()
# tf:
# tfidf_vectorizer = CountVectorizer()

# tfidf:
# tfidf_vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words= 'english')

# uni-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,1),token_pattern=r'\b\w+\b', min_df=1)

# bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(2,2),token_pattern=r'\b\w+\b', min_df=1)

# tri-gram:
tfidf_vectorizer = CountVectorizer(ngram_range=(3,3),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram & bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram,bi-gram & tri-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,3),token_pattern=r'\b\w+\b', min_df=1)


X_train_tfidf = tfidf_vectorizer.fit_transform(train_X)
vocabulary = tfidf_vectorizer.get_feature_names()

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tfidf.shape)

Time taken to extract features from training data : 0.801957 seconds
n_samples: 13525, n_features: 135916


C:\Users\nabil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
t = time()
X_test_tfidf = tfidf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tfidf.shape)

Time taken to extract features from test data : 0.061434 seconds
n_samples: 3382, n_features: 135916


In [ ]:
pd.DataFrame(data=X_test_tfidf.toarray(), columns = vocabulary).iloc[:,0::2]

,0 an hour,0 in their,0 patience for,000 000 bc,000 civilians and,000 has been,000 isis sites,000 islamic terrorist,000 per year,000 s of,...,zombies heading towards,zone please cut,zones are exactly,zones are there,zooming in on,zormixar is strategic,zucchini fritters are,zzzz call me,à la sqlite,ايلي ايليا twi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x = pd.DataFrame(data=X_test_tfidf.toarray(), columns = vocabulary).iloc[:,0::1]
x = x[(x > 0).any(axis=1)]
x

,0 an hour,0 and shut,0 in their,0 men on,0 patience for,00 on this,000 000 bc,000 children work,000 civilians and,000 foreign recruits,...,zormixar is strategic,zozo strangers that,zucchini fritters are,zynga for being,zzzz call me,½ the weight,à la sqlite,الرصافة neighborhood in,ايلي ايليا twi,ايليا twi insta
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# build naive bayes classification model
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)


train time: 0.057s


In [ ]:
# predict the new document from the testing dataset
print("Naive Bayes : \n")

t = time()
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

test_time = time() - t
# print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))




Naive Bayes : 

accuracy:   0.767
precision:   0.752
recall:   0.583
f1:   0.632
              precision    recall  f1-score   support

      racism       0.75      0.40      0.52       395
      sexism       0.74      0.42      0.53       686
     neither       0.77      0.94      0.85      2301

    accuracy                           0.77      3382
   macro avg       0.75      0.58      0.63      3382
weighted avg       0.76      0.77      0.74      3382



In [ ]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]}

bernoulli_nb_grid = GridSearchCV(naive_bayes_classifier, param_grid=params, n_jobs=-1, cv=5, verbose=5)
bernoulli_nb_grid.fit(X_train_tfidf, train_y)

print('Train Accuracy : %.3f'%bernoulli_nb_grid.best_estimator_.score(X_train_tfidf, train_y))
print('Test Accuracy : %.3f'%bernoulli_nb_grid.best_estimator_.score(X_test_tfidf, test_y))
print('Best Accuracy Through Grid Search : %.3f'%bernoulli_nb_grid.best_score_)
print('Best Parameters : ',bernoulli_nb_grid.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Train Accuracy : 0.897
Test Accuracy : 0.742
Best Accuracy Through Grid Search : 0.741
Best Parameters :  {'alpha': 10.0}


In [ ]:
#SVC
from sklearn.svm import SVC

clf = SVC(C=1.0, kernel='linear', gamma = 'auto').fit(X_train_tfidf, train_y)


In [ ]:
y_pred = clf.predict(X_test_tfidf)
print("Support Vector Machine : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Support Vector Machine : 

accuracy:   0.750
precision:   0.811
recall:   0.486
f1:   0.524
              precision    recall  f1-score   support

      racism       0.82      0.15      0.26       395
      sexism       0.87      0.33      0.47       686
     neither       0.74      0.98      0.84      2301

    accuracy                           0.75      3382
   macro avg       0.81      0.49      0.52      3382
weighted avg       0.78      0.75      0.70      3382



In [ ]:
# ## TAKES A LONG TIME TO EXECUTE


# param_grid = {'C': [0.1, 1, 10, 100, 1000],
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf']}

# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# # fitting the model for grid search
# grid.fit(X_train_tfidf, train_y)

# # print best parameter after tuning
# print(grid.best_params_)

# # print how our model looks after hyper-parameter tuning
# print(grid.best_estimator_)

# grid_predictions = grid.predict(X_test_tfidf)




In [ ]:
clf = SVC(C=100, kernel='rbf', gamma = 0.001).fit(X_train_tfidf, train_y)
y_pred = clf.predict(X_test_tfidf)

print("Support Vector Machine After Grid Search : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Support Vector Machine After Grid Search : 

accuracy:   0.747
precision:   0.820
recall:   0.474
f1:   0.508
              precision    recall  f1-score   support

      racism       0.84      0.13      0.22       395
      sexism       0.89      0.31      0.46       686
     neither       0.73      0.98      0.84      2301

    accuracy                           0.75      3382
   macro avg       0.82      0.47      0.51      3382
weighted avg       0.78      0.75      0.69      3382



In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier


clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,train_y)


In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(X_test_tfidf)
print("Decision Tree : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Decision Tree : 

accuracy:   0.741
precision:   0.701
recall:   0.525
f1:   0.567
              precision    recall  f1-score   support

      racism       0.62      0.29      0.40       395
      sexism       0.73      0.35      0.47       686
     neither       0.75      0.94      0.83      2301

    accuracy                           0.74      3382
   macro avg       0.70      0.53      0.57      3382
weighted avg       0.73      0.74      0.71      3382



In [ ]:
# Takes time to execute Grid Search

# param_dict = {"criterion" : ['gini', 'entropy'],
#               "max_depth" : range(1,10),
#               "min_samples_split" : range(1,10),
#               "min_samples_leaf" : range(1,5)
# }

# grid = GridSearchCV( DecisionTreeClassifier(), param_grid = param_dict,
#                     cv = 10, verbose = 1, n_jobs = -1)

# grid.fit(X_train_tfidf,train_y)

# print(grid.best_params_)
# print(grid.best_estimator_)
# print(grid.best_score_)


In [ ]:
# After Grid search: best parameters

clf = DecisionTreeClassifier(max_depth=9, min_samples_leaf=3, min_samples_split=8)
print("Decision Tree After Grid Search: \n")

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,train_y)
y_pred = clf.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Decision Tree After Grid Search: 

accuracy:   0.738
precision:   0.867
recall:   0.444
f1:   0.465
              precision    recall  f1-score   support

      racism       0.92      0.09      0.16       395
      sexism       0.96      0.25      0.40       686
     neither       0.72      1.00      0.84      2301

    accuracy                           0.74      3382
   macro avg       0.87      0.44      0.46      3382
weighted avg       0.79      0.74      0.67      3382



In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100)

clf.fit(X_train_tfidf, train_y)



RandomForestClassifier()

In [ ]:
# performing predictions on the test dataset
y_pred = clf.predict(X_test_tfidf)
print("Random Forest Classifier : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Random Forest Classifier : 

accuracy:   0.744
precision:   0.822
recall:   0.475
f1:   0.513
              precision    recall  f1-score   support

      racism       0.86      0.17      0.29       395
      sexism       0.87      0.27      0.41       686
     neither       0.73      0.98      0.84      2301

    accuracy                           0.74      3382
   macro avg       0.82      0.48      0.51      3382
weighted avg       0.78      0.74      0.69      3382



In [ ]:
# takes a long time to execute Grid Search

# param_grid = {
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

# CV_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv= 5)
# CV_rfc.fit(X_train_tfidf, train_y)
# print(CV_rfc.best_params_)
# print(CV_rfc.best_estimator_)
# print(CV_rfc.best_score_)


In [ ]:
clf = RandomForestClassifier(criterion= 'gini',max_depth=4, n_estimators=200, random_state=42)
print("Random Forest Classifier After Grid Search : \n")

clf.fit(X_train_tfidf, train_y)

y_pred = clf.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)


print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Random Forest Classifier After Grid Search : 

accuracy:   0.680
precision:   0.227
recall:   0.333
f1:   0.270
              precision    recall  f1-score   support

      racism       0.00      0.00      0.00       395
      sexism       0.00      0.00      0.00       686
     neither       0.68      1.00      0.81      2301

    accuracy                           0.68      3382
   macro avg       0.23      0.33      0.27      3382
weighted avg       0.46      0.68      0.55      3382



C:\Users\nabil\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nabil\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nabil\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nabil\anaconda3\lib\site-packages\sklearn\metrics\_classifi

In [ ]:
#logistic Regression
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(max_iter = 1000)

# fit the model with data
logreg.fit(X_train_tfidf,train_y)


LogisticRegression(max_iter=1000)

In [ ]:
y_pred=logreg.predict(X_test_tfidf)
print("Logistic Regression : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Logistic Regression : 

accuracy:   0.749
precision:   0.823
recall:   0.476
f1:   0.509
              precision    recall  f1-score   support

      racism       0.84      0.12      0.21       395
      sexism       0.89      0.32      0.47       686
     neither       0.74      0.98      0.84      2301

    accuracy                           0.75      3382
   macro avg       0.82      0.48      0.51      3382
weighted avg       0.78      0.75      0.69      3382



In [ ]:
# takes a long time to execute Grid Search

# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
# logreg=LogisticRegression()
# logreg_cv=GridSearchCV(logreg,grid,cv=10)
# logreg_cv.fit(X_train_tfidf,train_y)

# print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
# print(logreg_cv.best_estimator_)
# print("accuracy :",logreg_cv.best_score_)



In [ ]:
clf = LogisticRegression(C=10.0, penalty = 'l2', max_iter = 1000)
print("Logistic Regression After Grid Search: \n")

clf.fit(X_train_tfidf, train_y)

y_pred = clf.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Logistic Regression After Grid Search: 

accuracy:   0.752
precision:   0.821
recall:   0.486
f1:   0.525
              precision    recall  f1-score   support

      racism       0.84      0.15      0.25       395
      sexism       0.88      0.33      0.48       686
     neither       0.74      0.98      0.84      2301

    accuracy                           0.75      3382
   macro avg       0.82      0.49      0.52      3382
weighted avg       0.78      0.75      0.70      3382



In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=4)

# Train the model using the training sets
model.fit(X_train_tfidf,train_y)


KNeighborsClassifier(n_neighbors=4)

In [ ]:
y_pred = model.predict(X_test_tfidf)
print("K Nearest Neighbors : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

K Nearest Neighbors : 

accuracy:   0.687
precision:   0.784
recall:   0.352
f1:   0.310
              precision    recall  f1-score   support

      racism       1.00      0.03      0.05       395
      sexism       0.67      0.03      0.07       686
     neither       0.69      1.00      0.81      2301

    accuracy                           0.69      3382
   macro avg       0.78      0.35      0.31      3382
weighted avg       0.72      0.69      0.57      3382



C:\Users\nabil\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [ ]:
# takes a long time to execute Grid Search

# k_range = list(range(1, 31))
# param_grid = dict(n_neighbors=k_range)

# # defining parameter range
# grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)

# # fitting the model for grid search
# grid_search=grid.fit(X_train_tfidf,train_y)

# print(grid_search.best_params_)
# print(grid_search.best_estimator_)
# print(grid_search.best_score_)


In [ ]:
model = KNeighborsClassifier(n_neighbors=6)
print("K Nearest Neighbors After Grid Search: \n")

model.fit(X_train_tfidf, train_y)

y_pred = model.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

K Nearest Neighbors After Grid Search: 

accuracy:   0.683
precision:   0.728
recall:   0.342
f1:   0.288
              precision    recall  f1-score   support

      racism       1.00      0.02      0.04       395
      sexism       0.50      0.01      0.01       686
     neither       0.68      1.00      0.81      2301

    accuracy                           0.68      3382
   macro avg       0.73      0.34      0.29      3382
weighted avg       0.68      0.68      0.56      3382



C:\Users\nabil\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# UNDERSAMPLED DATA


In [ ]:
df_final["Annotation"].value_counts()

2    11501
1     3430
0     1976
Name: Annotation, dtype: int64

In [ ]:
class_3,class_2,class_1 = df_final.Annotation.value_counts()
c3 = df_final[df_final['Annotation'] == '2']
c2 = df_final[df_final['Annotation'] == '1']
c1 = df_final[df_final['Annotation'] == '0']
df_3 = c3.sample(class_1)
df_2 = c2.sample(class_1)

In [ ]:
undersampled_df = pd.concat([df_3,df_2,c1],axis=0)

In [ ]:
undersampled_df["Annotation"].value_counts()

2    1976
1    1976
0    1976
Name: Annotation, dtype: int64

In [ ]:
train_X,test_X, train_y, test_y = train_test_split(undersampled_df['text'], undersampled_df['Annotation'],
                                                   test_size = 0.2, random_state = 10, stratify = undersampled_df['Annotation'])

In [ ]:
t = time()
# tf:
# tfidf_vectorizer = CountVectorizer()

# tfidf:
# tfidf_vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words= 'english')

# uni-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,1),token_pattern=r'\b\w+\b', min_df=1)

# bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(2,2),token_pattern=r'\b\w+\b', min_df=1)

# tri-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(3,3),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram & bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram,bi-gram & tri-gram:
tfidf_vectorizer = CountVectorizer(ngram_range=(1,3),token_pattern=r'\b\w+\b', min_df=1)


X_train_tfidf = tfidf_vectorizer.fit_transform(train_X)
vocabulary = tfidf_vectorizer.get_feature_names()

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tfidf.shape)

Time taken to extract features from training data : 0.486571 seconds
n_samples: 4742, n_features: 100952


C:\Users\nabil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
t = time()
X_test_tfidf = tfidf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tfidf.shape)

Time taken to extract features from test data : 0.039909 seconds
n_samples: 1186, n_features: 100952


In [ ]:
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)


train time: 0.016s


In [ ]:
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tfidf)
print("Naive Bayes : \n")

test_time = time() - t
# print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))




Naive Bayes : 

accuracy:   0.759
precision:   0.759
recall:   0.759
f1:   0.747
              precision    recall  f1-score   support

      racism       0.76      0.94      0.84       395
      sexism       0.76      0.82      0.79       396
     neither       0.76      0.51      0.61       395

    accuracy                           0.76      1186
   macro avg       0.76      0.76      0.75      1186
weighted avg       0.76      0.76      0.75      1186



In [ ]:
clf = SVC(C=100, kernel='rbf', gamma = 0.001).fit(X_train_tfidf, train_y)
y_pred = clf.predict(X_test_tfidf)
print("Support Vector Machine : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Support Vector Machine : 

accuracy:   0.758
precision:   0.792
recall:   0.758
f1:   0.763
              precision    recall  f1-score   support

      racism       0.89      0.77      0.83       395
      sexism       0.87      0.66      0.75       396
     neither       0.61      0.84      0.71       395

    accuracy                           0.76      1186
   macro avg       0.79      0.76      0.76      1186
weighted avg       0.79      0.76      0.76      1186



In [ ]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,train_y)
print("Decision Tree : \n")

#Predict the response for test dataset
y_pred = clf.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Decision Tree : 

accuracy:   0.714
precision:   0.722
recall:   0.714
f1:   0.716
              precision    recall  f1-score   support

      racism       0.81      0.78      0.79       395
      sexism       0.75      0.67      0.71       396
     neither       0.60      0.69      0.65       395

    accuracy                           0.71      1186
   macro avg       0.72      0.71      0.72      1186
weighted avg       0.72      0.71      0.72      1186



In [ ]:
clf = RandomForestClassifier(n_estimators = 100)

clf.fit(X_train_tfidf, train_y)

y_pred = clf.predict(X_test_tfidf)

print("Random Forest Classifier : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Random Forest Classifier : 

accuracy:   0.749
precision:   0.782
recall:   0.749
f1:   0.751
              precision    recall  f1-score   support

      racism       0.85      0.82      0.84       395
      sexism       0.89      0.61      0.72       396
     neither       0.60      0.82      0.69       395

    accuracy                           0.75      1186
   macro avg       0.78      0.75      0.75      1186
weighted avg       0.78      0.75      0.75      1186



In [ ]:
logreg = LogisticRegression(max_iter=1000)

# fit the model with data
logreg.fit(X_train_tfidf,train_y)

y_pred=logreg.predict(X_test_tfidf)

print("Logistic Regression : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Logistic Regression : 

accuracy:   0.788
precision:   0.797
recall:   0.788
f1:   0.790
              precision    recall  f1-score   support

      racism       0.86      0.86      0.86       395
      sexism       0.85      0.73      0.78       396
     neither       0.68      0.77      0.72       395

    accuracy                           0.79      1186
   macro avg       0.80      0.79      0.79      1186
weighted avg       0.80      0.79      0.79      1186



In [ ]:
model = KNeighborsClassifier(n_neighbors=6)

# Train the model using the training sets
model.fit(X_train_tfidf,train_y)

y_pred = model.predict(X_test_tfidf)

print("K Nearest Neighbors : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

K Nearest Neighbors : 

accuracy:   0.387
precision:   0.677
recall:   0.387
f1:   0.280
              precision    recall  f1-score   support

      racism       0.93      0.04      0.07       395
      sexism       0.74      0.15      0.26       396
     neither       0.35      0.97      0.52       395

    accuracy                           0.39      1186
   macro avg       0.68      0.39      0.28      1186
weighted avg       0.68      0.39      0.28      1186



C:\Users\nabil\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# OVERSAMPLED DATA


In [ ]:
df_1 = c1.sample(class_3, replace=True)
df_2_over = c2.sample(class_3, replace=True)

In [ ]:
class_3

11501

In [ ]:

oversampled_df = pd.concat([c3,df_1,df_2_over], axis=0)

In [ ]:
oversampled_df["Annotation"].value_counts()

2    11501
0    11501
1    11501
Name: Annotation, dtype: int64

In [ ]:
oversampled_df

,text,Annotation
5406,Someone is going home #mkr ...that obviously cannot cook,2
5407,They didn t even wash the chicken #MKR,2
5408,#MKR Is honestly so fucking staged. The most over rated show after #ImACelebrityAU #MKR2015,2
5409,Can someone smash that bottle of Rose amp Lime Cordial over Drasko s head please #MKR,2
5410,"Will someone pls assist Colin in the washing of his hair. Sorry Colin, big fan but pls... Some shampoo mate #mkr",2
...,...,...
5015,"There are far more bigger problems than sexism out there in the world, I m not sexist and none of my friends are either",1
4239,"xbox s were designed for boys and make up was designed for girls, get the idea #notsexist #promise #lol",1
2961,Opt in solves these problems.,1
2261,#MKR Tart. Lol.,1


In [ ]:
train_X,test_X, train_y, test_y = train_test_split(oversampled_df['text'], oversampled_df['Annotation'],
                                                   test_size = 0.2, random_state = 10, stratify = oversampled_df['Annotation'])

In [ ]:
t = time()
# tf:
# tfidf_vectorizer = CountVectorizer()

# tfidf:
# tfidf_vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words= 'english')

# uni-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,1),token_pattern=r'\b\w+\b', min_df=1)

# bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(2,2),token_pattern=r'\b\w+\b', min_df=1)

# tri-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(3,3),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram & bi-gram:
# tfidf_vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b', min_df=1)

# uni-gram,bi-gram & tri-gram:
tfidf_vectorizer = CountVectorizer(ngram_range=(1,3),token_pattern=r'\b\w+\b', min_df=1)


X_train_tfidf = tfidf_vectorizer.fit_transform(train_X)
vocabulary = tfidf_vectorizer.get_feature_names()

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tfidf.shape)

Time taken to extract features from training data : 1.858445 seconds
n_samples: 27602, n_features: 252252


C:\Users\nabil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
t = time()
X_test_tfidf = tfidf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tfidf.shape)

Time taken to extract features from test data : 0.245707 seconds
n_samples: 6901, n_features: 252252


In [ ]:
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)


train time: 0.068s


In [ ]:
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tfidf)
print("Naive Bayes : \n")
test_time = time() - t
# print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))


Naive Bayes : 

accuracy:   0.888
precision:   0.899
recall:   0.888
f1:   0.882
              precision    recall  f1-score   support

      racism       0.85      1.00      0.92      2300
      sexism       0.87      0.98      0.92      2301
     neither       0.98      0.69      0.81      2300

    accuracy                           0.89      6901
   macro avg       0.90      0.89      0.88      6901
weighted avg       0.90      0.89      0.88      6901



In [ ]:
clf = SVC(C=100, kernel='rbf', gamma = 0.001).fit(X_train_tfidf, train_y)
y_pred = clf.predict(X_test_tfidf)
print("Support Vector Machine : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Support Vector Machine : 

accuracy:   0.962
precision:   0.962
recall:   0.962
f1:   0.961
              precision    recall  f1-score   support

      racism       0.98      1.00      0.99      2300
      sexism       0.95      0.96      0.96      2301
     neither       0.96      0.92      0.94      2300

    accuracy                           0.96      6901
   macro avg       0.96      0.96      0.96      6901
weighted avg       0.96      0.96      0.96      6901



In [ ]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,train_y)
print("Decision Tree : \n")

#Predict the response for test dataset
y_pred = clf.predict(X_test_tfidf)

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Decision Tree : 

accuracy:   0.925
precision:   0.928
recall:   0.925
f1:   0.923
              precision    recall  f1-score   support

      racism       0.92      1.00      0.96      2300
      sexism       0.89      0.97      0.93      2301
     neither       0.97      0.81      0.88      2300

    accuracy                           0.93      6901
   macro avg       0.93      0.93      0.92      6901
weighted avg       0.93      0.93      0.92      6901



In [ ]:
clf = RandomForestClassifier(n_estimators = 100)

clf.fit(X_train_tfidf, train_y)

y_pred = clf.predict(X_test_tfidf)

print("Random Forest Classifier : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Random Forest Classifier : 

accuracy:   0.970
precision:   0.969
recall:   0.970
f1:   0.969
              precision    recall  f1-score   support

      racism       0.98      1.00      0.99      2300
      sexism       0.97      0.96      0.97      2301
     neither       0.96      0.95      0.95      2300

    accuracy                           0.97      6901
   macro avg       0.97      0.97      0.97      6901
weighted avg       0.97      0.97      0.97      6901



In [ ]:
logreg = LogisticRegression(max_iter=1000)

# fit the model with data
logreg.fit(X_train_tfidf,train_y)

y_pred=logreg.predict(X_test_tfidf)

print("Logistic Regression : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

Logistic Regression : 

accuracy:   0.958
precision:   0.958
recall:   0.958
f1:   0.958
              precision    recall  f1-score   support

      racism       0.96      1.00      0.98      2300
      sexism       0.95      0.97      0.96      2301
     neither       0.97      0.90      0.93      2300

    accuracy                           0.96      6901
   macro avg       0.96      0.96      0.96      6901
weighted avg       0.96      0.96      0.96      6901



In [ ]:
model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(X_train_tfidf,train_y)

y_pred = model.predict(X_test_tfidf)

print("K Nearest Neighbors : \n")

score1 = metrics.accuracy_score(test_y, y_pred)
score2 = metrics.f1_score(test_y, y_pred,average='macro')
score3 = metrics.precision_score(test_y, y_pred, average='macro')
score4 = metrics.recall_score(test_y, y_pred, average='macro')
print("accuracy:   %0.3f" % score1)
print("precision:   %0.3f" % score3)
print("recall:   %0.3f" % score4)
print("f1:   %0.3f" % score2)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['racism', 'sexism', 'neither']))

C:\Users\nabil\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K Nearest Neighbors : 

accuracy:   0.907
precision:   0.910
recall:   0.907
f1:   0.908
              precision    recall  f1-score   support

      racism       0.99      0.97      0.98      2300
      sexism       0.91      0.84      0.87      2301
     neither       0.83      0.91      0.87      2300

    accuracy                           0.91      6901
   macro avg       0.91      0.91      0.91      6901
weighted avg       0.91      0.91      0.91      6901



In [ ]:
duplicate = oversampled_df[oversampled_df.duplicated()]

In [ ]:
duplicate[duplicate["Annotation"]=='1']

,text,Annotation
3106,#QuestionsForMen Are you aware that feminism is teaching women that men cannot experience sexism or discrimination ht,1
2878,"40 year old woman marries herself after she can t find Mr. Right, goes on honeymoon to Dubai lollol omg stop",1
5019,your right im a wanker. But I don t even have enough space in five tweets to point out how much of a cu,1
4321,I m not sexist but fuck me half of the women yids on Twitter are fucking clueless,1
2082,#MKR anyone can cook from a can girls.,1
...,...,...
2764,Why don t the like me Andre #mkr. #cuntfacebitch,1
4239,"xbox s were designed for boys and make up was designed for girls, get the idea #notsexist #promise #lol",1
2961,Opt in solves these problems.,1
2261,#MKR Tart. Lol.,1


In [ ]:
df_final[df_final["Annotation"]=='0']

,text,Annotation
0,So Drasko just said he was impressed the girls cooked half a chicken.. They cooked a whole one #MKR,0
1,Drasko they didn t cook half a bird you idiot #mkr,0
2,Hopefully someone cooks Drasko in the next ep of #MKR,0
3,of course you were born in serbia...you re as fucked as A Serbian Film #MKR,0
4,"These girls are the equivalent of the irritating Asian girls a couple years ago. Well done, 7. #MKR",0
...,...,...
1971,Drasko they didn t cook half a bird you idiot #mkr,0
1972,Hopefully someone cooks Drasko in the next ep of #MKR,0
1973,#MKR Lost the plot - where s the big Texan with the elephant sized steaks that they all have for brekkie,0
1974,It hasn t been a good few weeks for #ISIS. A new front has opened up in #Sinjar and they re about to lose the battle f,0


In [ ]:
df_final[df_final["Annotation"]=='2']

,text,Annotation
5406,Someone is going home #mkr ...that obviously cannot cook,2
5407,They didn t even wash the chicken #MKR,2
5408,#MKR Is honestly so fucking staged. The most over rated show after #ImACelebrityAU #MKR2015,2
5409,Can someone smash that bottle of Rose amp Lime Cordial over Drasko s head please #MKR,2
5410,"Will someone pls assist Colin in the washing of his hair. Sorry Colin, big fan but pls... Some shampoo mate #mkr",2
...,...,...
16902,"BillSpindle: It s all about power at the top, but for fighters on the ground #Tikrit is religious war on both sides",2
16903,"Iraq: #ISIS sets off 21 car bombs in Anbar as usual, randomly killing amp maiming innocents. htt",2
16904,DEAR STATE DEPA MENT: WHERE IS HILLARY CLINTON S OF-109 FORM #HillaryEmail #tcot #PJNET,2
16905,Let s go,2


In [ ]:
df_final[df_final["Annotation"]=='1']

,text,Annotation
1976,These two are revolting #MKR #MKR2015,1
1977,#katieandnikki stop calling yourselves pretty and hot..you re not and saying it a million times doesn t make you either...STFU #MKR,1
1978,"If keep these girls in the comp, which I suspect they will, I ll never watch #MKR again.\n\nSO SO SO Rigged.",1
1979,The menus look like they were made by a 5 year old little girl...in this case just the mental age of a 5 year old girl I guess #MKR,1
1980,Wish these blondes were in that How To Get Away With Murder show....#MKR,1
...,...,...
5401,"Katie and Nikki, smug, vacuous, condescending, putrid, self congratulatory and bland. There. #MKR",1
5402,I can barely watch the #MKR episode of Katie and Nikki or whatever. Like my skin is crawling. They have thattt many tickets on themselves,1
5403,Really Really Another round #MKR those butcher promo girls better be nice to Sheri and Emilie. Yes OR else,1
5404,#MKR you d think in her downtime Annie would have paid Napoléon Perdis a visit and learnt not to use the same coloured blush as your hair...,1


In [ ]:
df_final["Annotation"].value_counts()

2    11501
1     3430
0     1976
Name: Annotation, dtype: int64

In [ ]:
df_final.shape

(16907, 2)